# PRFs for subject LL collected 20210810, analyzed using popeye

# Setup

In [1]:
import os, pickle, glob
import pathlib
from bids import BIDSLayout

import numpy as np

import matplotlib.pyplot as plt

import nibabel as nib

import nilearn
from nilearn.input_data import NiftiMasker
from nilearn.image import get_data, math_img, index_img, load_img, concat_imgs

import utils

from PIL import Image
import cv2 as cv

In [2]:
print(' numpy: ', np.__version__, '\n',
      'nibabel: ', nib.__version__, '\n',
      'nilearn: ', nilearn.__version__, '\n')

 numpy:  1.20.0 
 nibabel:  3.2.1 
 nilearn:  0.7.0 



In [3]:
sub = "LL"
ses = "20210810"

root_dir = "/Users/smerdis/data/LGN/BIDS/LL_combined"
raw_data_dir = os.path.abspath(f"{root_dir}/sub-{sub}/")
raw_layout = BIDSLayout(raw_data_dir, validate=False, derivatives=False)
derivs_dir = os.path.abspath(f"{root_dir}/derivatives/prf")
freesurfer_dir = os.path.abspath(f"{root_dir}/derivatives/freesurfer")

out_dir = os.path.abspath(f"{derivs_dir}/sub-{sub}")
if not os.path.exists(out_dir): # if we haven't preprocessed the LGNpRF data yet
    pathlib.Path(out_dir).mkdir(parents=True, exist_ok=False)
    # Go to Preprocessing section and mcflirt the runs etc

//anaconda/envs/streams/lib/python3.7/site-packages/bids/layout/models.py:152: FutureWarning: The 'extension' entity currently excludes the leading dot ('.'). As of version 0.14.0, it will include the leading dot. To suppress this warning and include the leading dot, use `bids.config.set_option('extension_initial_dot', True)`.
  FutureWarning)


In [4]:
!ls {out_dir}

20210831-111808
20210907-113646
20210908-081328
Screenshots
bigstack-LL-AB.pkl
bigstack-LL-B.pkl
bigstack-LL.pkl
ses-20210810
sub-LL_ses-20210810_desc-wholebrainmask-back40_roi.nii.gz
sub-LL_ses-20210810_desc-wholebrainmask-back_roi.nii.gz
sub-LL_ses-20210810_mask.nii.gz
sub-LL_ses-20210810_mcflirt_mean_trim.nii
sub-LL_ses-20210810_mcflirt_mean_trim_evenrunsonly.nii
sub-LL_ses-20210810_mcflirt_mean_trim_oddevenconcat.nii
sub-LL_ses-20210810_mcflirt_mean_trim_oddrunsonly.nii
sub-LL_ses-20210810_refvol.json
sub-LL_ses-20210810_refvol.nii.gz


In [5]:
preproc_layout = BIDSLayout(out_dir, validate=False)

anat_file = sorted([f for f in raw_layout.get(subject=sub, session=ses, suffix='T1w',
            extension=['nii.gz'], return_type='file')])[0]
print(f"anat file: {anat_file}")
raw_bolds_prf = sorted([f for f in raw_layout.get(subject=sub, session=ses, task='prf', suffix='bold',
            extension=['nii.gz'], return_type='file')])
mcf_bolds_prf = sorted([f for f in preproc_layout.get(subject=sub, session=ses, task='prf', suffix='bold',
            extension=['nii.gz'], return_type='file') if 'mcflirt' in f])

prf_mask_file = os.path.abspath(f"{out_dir}/sub-{sub}_ses-{ses}_mask.nii.gz")

anat file: /Users/smerdis/data/LGN/BIDS/LL_combined/sub-LL/ses-20210810/anat/sub-LL_ses-20210810_T1w.nii.gz


In [7]:
prf_ref_vol_noext = f"sub-{sub}_ses-{ses}_refvol"
prf_ref_vol_name = f"{prf_ref_vol_noext}.nii.gz"
prf_ref_vol_path = f"{out_dir}/{prf_ref_vol_name}"
print(prf_ref_vol_path)
!ls {prf_ref_vol_path}

/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/sub-LL_ses-20210810_refvol.nii.gz
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/sub-LL_ses-20210810_refvol.nii.gz


# Preprocessing

## Make reference volume from first dicom - make sure to change this

In [11]:
first_dicom_prf = "/Users/smerdis/Dropbox/data/LGN/Ll_Lgnprf_20210810/flat_dicoms/IM-0007-0001.dcm"

In [12]:
dcm2niix_cmd = f"dcm2niix -v 2 -s y -f {prf_ref_vol_noext} -z y -o {out_dir} {first_dicom_prf}"
print(dcm2niix_cmd)

dcm2niix -v 2 -s y -f sub-LL_ses-20210810_refvol -z y -o /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL /Users/smerdis/Dropbox/data/LGN/Ll_Lgnprf_20210810/flat_dicoms/IM-0007-0001.dcm


## Motion correct BOLD runs to this refvol

In [16]:
for this_epi in raw_bolds_prf:
    epi_stub_mcf = utils.change_bids_description(this_epi, 'desc-mcflirt')
    if not os.path.isdir(os.path.join(out_dir, f"ses-{ses}", "func")):
        os.makedirs(os.path.join(out_dir, f"ses-{ses}", "func"))
    full_outpath = os.path.join(out_dir, f"ses-{ses}", "func", epi_stub_mcf)
    mcflirt_cmd = f"mcflirt -reffile {prf_ref_vol_path} -mats -plots -report -cost mutualinfo -smooth 16 -in {this_epi} -o {full_outpath}"
    print(mcflirt_cmd)
    os.system(mcflirt_cmd)

mcflirt -reffile /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/sub-LL_ses-20210810_refvol.nii.gz -mats -plots -report -cost mutualinfo -smooth 16 -in /Users/smerdis/data/LGN/BIDS/LL_combined/sub-LL/ses-20210810/func/sub-LL_ses-20210810_task-prf_run-01_bold.nii.gz -o /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/ses-20210810/func/sub-LL_ses-20210810_task-prf_run-01_desc-mcflirt_bold
mcflirt -reffile /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/sub-LL_ses-20210810_refvol.nii.gz -mats -plots -report -cost mutualinfo -smooth 16 -in /Users/smerdis/data/LGN/BIDS/LL_combined/sub-LL/ses-20210810/func/sub-LL_ses-20210810_task-prf_run-02_bold.nii.gz -o /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/ses-20210810/func/sub-LL_ses-20210810_task-prf_run-02_desc-mcflirt_bold
mcflirt -reffile /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/sub-LL_ses-20210810_refvol.nii.gz -mats -plots -report -cost mutualinfo -smoo

## Make brainmask from first run

In [7]:
!ls {prf_mask_file}

/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/sub-LL_ses-20210810_mask.nii.gz


In [22]:
prf_masker = NiftiMasker(detrend=False, standardize=False, mask_strategy="epi", t_r=2.0)
prf_masker.fit(mcf_bolds_prf[0])
prf_masker.mask_img_.to_filename(prf_mask_file)
prf_mask_img = prf_masker.mask_img_
print(prf_mask_file, prf_mask_img.shape)

/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/sub-LL_ses-20210810_mask.nii.gz (128, 128, 19)


## Read in, highpass filter, normalize to psc, and save

In [ ]:
# m = NiftiMasker(mask_img=prf_mask_file, standardize='psc', detrend=True,
#                 high_pass=0.02, t_r=2, mask_strategy='epi', verbose=1)
# m = NiftiMasker(mask_img=prf_mask_file, standardize='psc', t_r=2,
#                 verbose=1) # no highpass
# preproc_desc = 'desc-preproc-nohighpass'
m = NiftiMasker(mask_img=prf_mask_file, standardize=False, t_r=2, high_pass=0.02,
                verbose=1) # highpass, no psc
preproc_desc = 'desc-preproc-highpassnopsc'

In [ ]:
for epi in mcf_bolds_prf:
    preproc_fn = utils.change_bids_description(epi, preproc_desc)
    print(preproc_fn)
    preproc_func_dir = os.path.join(out_dir, f"ses-{ses}", "func")
    if not os.path.isdir(preproc_func_dir):
        os.mkdir(preproc_func_dir)
    full_outpath = os.path.join(preproc_func_dir, preproc_fn)
    preproc_data = m.fit_transform(epi)
    preproc_img = m.inverse_transform(preproc_data)
    print(full_outpath, preproc_img.shape)
    preproc_img.to_filename(full_outpath)

In [ ]:
!ls {preproc_func_dir}

## Average

In [ ]:
preproc_desc

In [ ]:
preproc_bolds_prf = sorted([f for f in preproc_layout.get(subject=sub, session=ses, task='prf', suffix='bold',
            extension=['nii'], return_type='file') if preproc_desc in f])
print('\n'.join(preproc_bolds_prf))

In [ ]:
# have a look at the affine
print(load_img(preproc_bolds_prf[0]).affine)

In [ ]:
prf_input_img = utils.make_timeseries_for_prf(preproc_bolds_prf)

In [ ]:
prf_input_img.shape

In [ ]:
nib.save(prf_input_img, 
         f"{out_dir}/sub-{sub}_ses-{ses}_{preproc_desc}_mean_trim.nii")

## Try a different approach to see if this fixes prfs - just avg the motion-corrected runs, no conversion to % signal change, no highpass

In [55]:
mcf_bolds_prf[1::2]

['/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/ses-20210810/func/sub-LL_ses-20210810_task-prf_run-02_desc-mcflirt_bold.nii.gz',
 '/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/ses-20210810/func/sub-LL_ses-20210810_task-prf_run-04_desc-mcflirt_bold.nii.gz',
 '/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/ses-20210810/func/sub-LL_ses-20210810_task-prf_run-06_desc-mcflirt_bold.nii.gz',
 '/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/ses-20210810/func/sub-LL_ses-20210810_task-prf_run-08_desc-mcflirt_bold.nii.gz',
 '/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/ses-20210810/func/sub-LL_ses-20210810_task-prf_run-10_desc-mcflirt_bold.nii.gz']

In [56]:
prf_input_mcflirtonly = utils.make_timeseries_for_prf(mcf_bolds_prf[1::2]) # [::2] = odd runs = A stim
nib.save(prf_input_mcflirtonly, 
         f"{out_dir}/sub-{sub}_ses-{ses}_mcflirt_mean_trim_evenrunsonly.nii")

(128, 128, 19, 139)
(128, 128, 19, 139)
(128, 128, 19, 139)
(128, 128, 19, 139)
(128, 128, 19, 139)


In [62]:
!fslinfo "{out_dir}/sub-{sub}_ses-{ses}_mcflirt_mean_trim_oddrunsonly.nii"

data_type	FLOAT64
dim1		128
dim2		128
dim3		19
dim4		138
datatype	64
pixdim1		1.750000
pixdim2		1.750000
pixdim3		1.750000
pixdim4		1.000000
cal_max		0.000000
cal_min		0.000000
file_type	NIFTI-1+


In [66]:
!fslinfo "{out_dir}/sub-{sub}_ses-{ses}_mcflirt_mean_trim_evenrunsonly.nii"

data_type	FLOAT64
dim1		128
dim2		128
dim3		19
dim4		138
datatype	64
pixdim1		1.750000
pixdim2		1.750000
pixdim3		1.750000
pixdim4		1.000000
cal_max		0.000000
cal_min		0.000000
file_type	NIFTI-1+


In [67]:
prf_input_concat = concat_imgs(
    [f"{out_dir}/sub-{sub}_ses-{ses}_mcflirt_mean_trim_oddrunsonly.nii",
    f"{out_dir}/sub-{sub}_ses-{ses}_mcflirt_mean_trim_evenrunsonly.nii"])

In [70]:
prf_input_concat.shape

(128, 128, 19, 276)

In [71]:
nib.save(prf_input_concat, f"{out_dir}/sub-{sub}_ses-{ses}_mcflirt_mean_trim_oddevenconcat.nii")

# Make cortical mask

## Convert some freesurfer mgz files to nifti

In [ ]:
ribbon_mgz = f"{freesurfer_dir}/{sub}/mri/ribbon.mgz"
ribbon_nii = f"{out_dir}/ribbon.nii.gz"
!ls {ribbon_mgz}

In [ ]:
mriconv_cmd = f"mri_convert --in_type mgz --out_type nii --out_orientation RAS \
{freesurfer_dir}/{sub}/mri/brain.mgz {out_dir}/brain.nii.gz"
print(mriconv_cmd)
os.system(mriconv_cmd)
mriconv_cmd = f"mri_convert --in_type mgz --out_type nii --out_orientation RAS \
{freesurfer_dir}/{sub}/mri/wm.seg.mgz {out_dir}/wmseg.nii.gz"
print(mriconv_cmd)
os.system(mriconv_cmd)
mriconv_cmd = f"mri_convert --in_type mgz --out_type nii --out_orientation RAS \
{ribbon_mgz} {ribbon_nii}"
print(mriconv_cmd)
os.system(mriconv_cmd)

## Use FLIRT to register func ref vol to anat, save transform

In [ ]:
flirt_cmd = f"flirt -in {prf_ref_vol_path} -ref {out_dir}/brain.nii.gz -wmseg \
{out_dir}/wmseg.nii.gz -dof 6 -omat {out_dir}/func2brain.mat \
-out {out_dir}/mb3refvol2t1"
print(flirt_cmd)
os.system(flirt_cmd)

In [ ]:
# # Register Dev's EPI to the anat and save it
# flirt_cmd = f"flirt -in {out_dir}/5_epi_retino.nii -ref {out_dir}/brain.nii.gz -wmseg \
# {out_dir}/wmseg.nii.gz -dof 6 -omat {out_dir}/func2brain_dev.mat \
# -out {out_dir}/devepi2t1"
# print(flirt_cmd)
# os.system(flirt_cmd)

## Invert the transform and save it

In [ ]:
# For Dev's data
# invxfm_cmd = f"convert_xfm -omat {out_dir}/brain2func_dev.mat -inverse {out_dir}/func2brain_dev.mat"
# print(invxfm_cmd)
# os.system(invxfm_cmd)

In [ ]:
invxfm_cmd = f"convert_xfm -omat {out_dir}/brain2func.mat -inverse {out_dir}/func2brain.mat"
print(invxfm_cmd)
os.system(invxfm_cmd)

## Apply inverse transform to convert freesurfer outputs to functional space

In [ ]:
brain2func_cmd = f"flirt -ref {prf_ref_vol_path} -in {out_dir}/brain.nii.gz -out {out_dir}/brain_funcspace.nii.gz \
-init {out_dir}/brain2func.mat -applyxfm"
print(brain2func_cmd)
os.system(brain2func_cmd)

In [ ]:
# For Dev's data
# brain2func_cmd = f"flirt -ref {out_dir}/5_epi_retino.nii -in {out_dir}/brain.nii.gz -out {out_dir}/brain_funcspace_dev.nii.gz \
# -init {out_dir}/brain2func_dev.mat -applyxfm"
# print(brain2func_cmd)
# os.system(brain2func_cmd)

## Load ribbon and extract cortex only

In [ ]:
def make_func_parc_mask(ribbon_nii, parc_codes, out_fn):
    ribbon_img = load_img(ribbon_nii)
    ribbon_data = ribbon_img.get_fdata()
    cortex_mask = np.zeros_like(ribbon_data)
    print(np.count_nonzero(cortex_mask))
    for c in parc_codes:
        cortex_mask[ribbon_data==c] = 1
    print(np.count_nonzero(cortex_mask))
    cortex_mask_img = nib.Nifti1Image(cortex_mask, ribbon_img.affine)
    nib.save(cortex_mask_img, out_fn)
    cmd = f"flirt -ref {prf_ref_vol_path} -in {out_fn} -out {out_fn}_funcspace.nii.gz -init {out_dir}/brain2func.mat -applyxfm"
    print(cmd)
    os.system(cmd)

In [ ]:
make_func_parc_mask(ribbon_nii, [3, 42], f"{out_dir}/cortex_mask")

In [ ]:
make_func_parc_mask(ribbon_nii, [3], f"{out_dir}/Lcortex_mask")
make_func_parc_mask(ribbon_nii, [42], f"{out_dir}/Rcortex_mask")

In [ ]:
ribbon2func_cmd = f"flirt -ref {prf_ref_vol_path} -in {out_dir}/ribbon.nii.gz -out {out_dir}/ribbon_funcspace.nii.gz \
-init {out_dir}/brain2func.mat -applyxfm"
print(ribbon2func_cmd)
os.system(ribbon2func_cmd)

In [ ]:
print(f"fsleyes {anat_file} {out_dir}/wmseg.nii.gz {out_dir}/ribbon.nii.gz")


# Screenshots to make prfs better

In [73]:
# sshots_dir = f"{out_dir}/Screenshots"
# imlist = [
#     f"{sshots_dir}/Sc_1LLTest_A_001.png",
#     f"{sshots_dir}/Sc_1LLTest_A_002.png",
#     f"{sshots_dir}/Sc_1LLTest_A_003.png",
#     f"{sshots_dir}/Sc_1LLTest_A_004.png",
#     f"{sshots_dir}/Sc_1LLTest_A_005.png",
#     f"{sshots_dir}/Sc_1LLTest_A_006.png",
#     f"{sshots_dir}/Sc_1LLTest_A_007.png",
#     f"{sshots_dir}/Blank.png",
#     f"{sshots_dir}/Sc_1LLTest_A_008.png",
#     f"{sshots_dir}/Sc_1LLTest_A_009.png",
#     f"{sshots_dir}/Sc_1LLTest_A_010.png",
#     f"{sshots_dir}/Sc_1LLTest_A_011.png",
#     f"{sshots_dir}/Sc_1LLTest_A_012.png",
#     f"{sshots_dir}/Sc_1LLTest_A_013.png",
#     f"{sshots_dir}/Sc_1LLTest_A_014.png",
#     f"{sshots_dir}/Blank.png",
#     f"{sshots_dir}/Sc_1LLTest_A_015.png",
#     f"{sshots_dir}/Sc_1LLTest_A_016.png",
#     f"{sshots_dir}/Sc_1LLTest_A_017.png",
#     f"{sshots_dir}/Sc_1LLTest_A_018.png",
#     f"{sshots_dir}/Sc_1LLTest_A_019.png",
#     f"{sshots_dir}/Sc_1LLTest_A_020.png",
#     f"{sshots_dir}/Sc_1LLTest_A_021.png",
#     f"{sshots_dir}/Blank.png",
#     f"{sshots_dir}/Sc_1LLTest_A_022.png",
#     f"{sshots_dir}/Sc_1LLTest_A_023.png",
#     f"{sshots_dir}/Sc_1LLTest_A_024.png",
#     f"{sshots_dir}/Sc_1LLTest_A_025.png",
#     f"{sshots_dir}/Sc_1LLTest_A_026.png",
#     f"{sshots_dir}/Sc_1LLTest_A_027.png",
#     f"{sshots_dir}/Sc_1LLTest_A_028.png",
#     f"{sshots_dir}/Blank.png",
#     f"{sshots_dir}/Sc_1LLTest_A_029.png",
#     f"{sshots_dir}/Sc_1LLTest_A_030.png",
#     f"{sshots_dir}/Sc_1LLTest_A_031.png",
#     f"{sshots_dir}/Sc_1LLTest_A_032.png",
#     f"{sshots_dir}/Sc_1LLTest_A_033.png",
#     f"{sshots_dir}/Sc_1LLTest_A_034.png",
#     f"{sshots_dir}/Sc_1LLTest_A_035.png",
#     f"{sshots_dir}/Blank.png",
#     f"{sshots_dir}/Sc_1LLTest_A_036.png",
#     f"{sshots_dir}/Sc_1LLTest_A_037.png",
#     f"{sshots_dir}/Sc_1LLTest_A_038.png",
#     f"{sshots_dir}/Sc_1LLTest_A_039.png",
#     f"{sshots_dir}/Sc_1LLTest_A_040.png",
#     f"{sshots_dir}/Sc_1LLTest_A_041.png",
#     f"{sshots_dir}/Sc_1LLTest_A_042.png",
#     f"{sshots_dir}/Blank.png",
#     f"{sshots_dir}/Sc_1LLTest_A_043.png",
#     f"{sshots_dir}/Sc_1LLTest_A_044.png",
#     f"{sshots_dir}/Sc_1LLTest_A_045.png",
#     f"{sshots_dir}/Sc_1LLTest_A_046.png",
#     f"{sshots_dir}/Sc_1LLTest_A_047.png",
#     f"{sshots_dir}/Sc_1LLTest_A_048.png",
#     f"{sshots_dir}/Sc_1LLTest_A_049.png",
#     f"{sshots_dir}/Blank.png",
#     f"{sshots_dir}/Sc_1LLTest_A_050.png",
#     f"{sshots_dir}/Sc_1LLTest_A_051.png",
#     f"{sshots_dir}/Sc_1LLTest_A_052.png",
#     f"{sshots_dir}/Sc_1LLTest_A_053.png",
#     f"{sshots_dir}/Sc_1LLTest_A_054.png",
#     f"{sshots_dir}/Sc_1LLTest_A_055.png",
#     f"{sshots_dir}/Sc_1LLTest_A_056.png",
#     f"{sshots_dir}/Blank.png",
#     f"{sshots_dir}/Sc_1LLTest_A_057.png",
#     f"{sshots_dir}/Sc_1LLTest_A_058.png",
#     f"{sshots_dir}/Sc_1LLTest_A_059.png",
#     f"{sshots_dir}/Sc_1LLTest_A_060.png",
#     f"{sshots_dir}/Sc_1LLTest_A_061.png",
#     f"{sshots_dir}/Sc_1LLTest_A_062.png",
#     f"{sshots_dir}/Sc_1LLTest_A_063.png",
#     f"{sshots_dir}/Blank.png",
#     f"{sshots_dir}/Sc_1LLTest_A_064.png",
#     f"{sshots_dir}/Sc_1LLTest_A_065.png",
#     f"{sshots_dir}/Sc_1LLTest_A_066.png",
#     f"{sshots_dir}/Sc_1LLTest_A_067.png",
#     f"{sshots_dir}/Sc_1LLTest_A_068.png",
#     f"{sshots_dir}/Sc_1LLTest_A_069.png",
#     f"{sshots_dir}/Sc_1LLTest_A_070.png",
#     f"{sshots_dir}/Blank.png",
#     f"{sshots_dir}/Sc_1LLTest_A_072.png",
#     f"{sshots_dir}/Sc_1LLTest_A_073.png",
#     f"{sshots_dir}/Sc_1LLTest_A_074.png",
#     f"{sshots_dir}/Sc_1LLTest_A_075.png",
#     f"{sshots_dir}/Sc_1LLTest_A_076.png",
#     f"{sshots_dir}/Sc_1LLTest_A_077.png",
#     f"{sshots_dir}/Sc_1LLTest_A_078.png",
#     f"{sshots_dir}/Blank.png",
#     f"{sshots_dir}/Sc_1LLTest_A_080.png",
#     f"{sshots_dir}/Sc_1LLTest_A_081.png",
#     f"{sshots_dir}/Sc_1LLTest_A_082.png",
#     f"{sshots_dir}/Sc_1LLTest_A_083.png",
#     f"{sshots_dir}/Sc_1LLTest_A_084.png",
#     f"{sshots_dir}/Sc_1LLTest_A_085.png",
#     f"{sshots_dir}/Sc_1LLTest_A_086.png",
#     f"{sshots_dir}/Blank.png",
#     f"{sshots_dir}/Sc_1LLTest_A_087.png",
#     f"{sshots_dir}/Sc_1LLTest_A_088.png",
#     f"{sshots_dir}/Sc_1LLTest_A_089.png",
#     f"{sshots_dir}/Sc_1LLTest_A_090.png",
#     f"{sshots_dir}/Sc_1LLTest_A_091.png",
#     f"{sshots_dir}/Sc_1LLTest_A_092.png",
#     f"{sshots_dir}/Sc_1LLTest_A_093.png",
#     f"{sshots_dir}/Blank.png",
#     f"{sshots_dir}/Sc_1LLTest_A_094.png",
#     f"{sshots_dir}/Sc_1LLTest_A_095.png",
#     f"{sshots_dir}/Sc_1LLTest_A_096.png",
#     f"{sshots_dir}/Sc_1LLTest_A_097.png",
#     f"{sshots_dir}/Sc_1LLTest_A_098.png",
#     f"{sshots_dir}/Sc_1LLTest_A_099.png",
#     f"{sshots_dir}/Sc_1LLTest_A_100.png",
#     f"{sshots_dir}/Blank.png",
#     f"{sshots_dir}/Sc_1LLTest_A_101.png",
#     f"{sshots_dir}/Sc_1LLTest_A_102.png",
#     f"{sshots_dir}/Sc_1LLTest_A_103.png",
#     f"{sshots_dir}/Sc_1LLTest_A_104.png",
#     f"{sshots_dir}/Sc_1LLTest_A_105.png",
#     f"{sshots_dir}/Sc_1LLTest_A_106.png",
#     f"{sshots_dir}/Sc_1LLTest_A_107.png",
#     f"{sshots_dir}/Blank.png",
#     f"{sshots_dir}/Sc_1LLTest_A_108.png",
#     f"{sshots_dir}/Sc_1LLTest_A_109.png",
#     f"{sshots_dir}/Sc_1LLTest_A_110.png",
#     f"{sshots_dir}/Sc_1LLTest_A_111.png",
#     f"{sshots_dir}/Sc_1LLTest_A_112.png",
#     f"{sshots_dir}/Sc_1LLTest_A_113.png",
#     f"{sshots_dir}/Sc_1LLTest_A_114.png",
#     f"{sshots_dir}/Blank.png",
# ]

In [79]:
imlist = [
    f"{sshots_dir}/Sc_1LLTest_B_001.png",
    f"{sshots_dir}/Sc_1LLTest_B_002.png",
    f"{sshots_dir}/Sc_1LLTest_B_003.png",
    f"{sshots_dir}/Sc_1LLTest_B_004.png",
    f"{sshots_dir}/Sc_1LLTest_B_005.png",
    f"{sshots_dir}/Sc_1LLTest_B_006.png",
    f"{sshots_dir}/Sc_1LLTest_B_007.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_1LLTest_B_008.png",
    f"{sshots_dir}/Sc_1LLTest_B_009.png",
    f"{sshots_dir}/Sc_1LLTest_B_010.png",
    f"{sshots_dir}/Sc_1LLTest_B_011.png",
    f"{sshots_dir}/Sc_1LLTest_B_012.png",
    f"{sshots_dir}/Sc_1LLTest_B_013.png",
    f"{sshots_dir}/Sc_1LLTest_B_014.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_1LLTest_B_015.png",
    f"{sshots_dir}/Sc_1LLTest_B_016.png",
    f"{sshots_dir}/Sc_1LLTest_B_017.png",
    f"{sshots_dir}/Sc_1LLTest_B_018.png",
    f"{sshots_dir}/Sc_1LLTest_B_019.png",
    f"{sshots_dir}/Sc_1LLTest_B_020.png",
    f"{sshots_dir}/Sc_1LLTest_B_021.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_1LLTest_B_022.png",
    f"{sshots_dir}/Sc_1LLTest_B_023.png",
    f"{sshots_dir}/Sc_1LLTest_B_024.png",
    f"{sshots_dir}/Sc_1LLTest_B_025.png",
    f"{sshots_dir}/Sc_1LLTest_B_026.png",
    f"{sshots_dir}/Sc_1LLTest_B_027.png",
    f"{sshots_dir}/Sc_1LLTest_B_028.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_1LLTest_B_029.png",
    f"{sshots_dir}/Sc_1LLTest_B_030.png",
    f"{sshots_dir}/Sc_1LLTest_B_031.png",
    f"{sshots_dir}/Sc_1LLTest_B_032.png",
    f"{sshots_dir}/Sc_1LLTest_B_033.png",
    f"{sshots_dir}/Sc_1LLTest_B_034.png",
    f"{sshots_dir}/Sc_1LLTest_B_035.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_1LLTest_B_036.png",
    f"{sshots_dir}/Sc_1LLTest_B_037.png",
    f"{sshots_dir}/Sc_1LLTest_B_038.png",
    f"{sshots_dir}/Sc_1LLTest_B_039.png",
    f"{sshots_dir}/Sc_1LLTest_B_040.png",
    f"{sshots_dir}/Sc_1LLTest_B_041.png",
    f"{sshots_dir}/Sc_1LLTest_B_042.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_1LLTest_B_043.png",
    f"{sshots_dir}/Sc_1LLTest_B_044.png",
    f"{sshots_dir}/Sc_1LLTest_B_045.png",
    f"{sshots_dir}/Sc_1LLTest_B_046.png",
    f"{sshots_dir}/Sc_1LLTest_B_047.png",
    f"{sshots_dir}/Sc_1LLTest_B_048.png",
    f"{sshots_dir}/Sc_1LLTest_B_049.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_1LLTest_B_050.png",
    f"{sshots_dir}/Sc_1LLTest_B_051.png",
    f"{sshots_dir}/Sc_1LLTest_B_052.png",
    f"{sshots_dir}/Sc_1LLTest_B_053.png",
    f"{sshots_dir}/Sc_1LLTest_B_054.png",
    f"{sshots_dir}/Sc_1LLTest_B_055.png",
    f"{sshots_dir}/Sc_1LLTest_B_056.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_1LLTest_B_057.png",
    f"{sshots_dir}/Sc_1LLTest_B_058.png",
    f"{sshots_dir}/Sc_1LLTest_B_059.png",
    f"{sshots_dir}/Sc_1LLTest_B_060.png",
    f"{sshots_dir}/Sc_1LLTest_B_061.png",
    f"{sshots_dir}/Sc_1LLTest_B_062.png",
    f"{sshots_dir}/Sc_1LLTest_B_063.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_1LLTest_B_064.png",
    f"{sshots_dir}/Sc_1LLTest_B_065.png",
    f"{sshots_dir}/Sc_1LLTest_B_066.png",
    f"{sshots_dir}/Sc_1LLTest_B_067.png",
    f"{sshots_dir}/Sc_1LLTest_B_068.png",
    f"{sshots_dir}/Sc_1LLTest_B_069.png",
    f"{sshots_dir}/Sc_1LLTest_B_070.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_1LLTest_B_071.png",
    f"{sshots_dir}/Sc_1LLTest_B_072.png",
    f"{sshots_dir}/Sc_1LLTest_B_073.png",
    f"{sshots_dir}/Sc_1LLTest_B_074.png",
    f"{sshots_dir}/Sc_1LLTest_B_075.png",
    f"{sshots_dir}/Sc_1LLTest_B_076.png",
    f"{sshots_dir}/Sc_1LLTest_B_077.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_1LLTest_B_078.png",
    f"{sshots_dir}/Sc_1LLTest_B_079.png",
    f"{sshots_dir}/Sc_1LLTest_B_080.png",
    f"{sshots_dir}/Sc_1LLTest_B_081.png",
    f"{sshots_dir}/Sc_1LLTest_B_082.png",
    f"{sshots_dir}/Sc_1LLTest_B_083.png",
    f"{sshots_dir}/Sc_1LLTest_B_084.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_1LLTest_B_085.png",
    f"{sshots_dir}/Sc_1LLTest_B_086.png",
    f"{sshots_dir}/Sc_1LLTest_B_087.png",
    f"{sshots_dir}/Sc_1LLTest_B_088.png",
    f"{sshots_dir}/Sc_1LLTest_B_089.png",
    f"{sshots_dir}/Sc_1LLTest_B_090.png",
    f"{sshots_dir}/Sc_1LLTest_B_091.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_1LLTest_B_092.png",
    f"{sshots_dir}/Sc_1LLTest_B_093.png",
    f"{sshots_dir}/Sc_1LLTest_B_094.png",
    f"{sshots_dir}/Sc_1LLTest_B_095.png",
    f"{sshots_dir}/Sc_1LLTest_B_096.png",
    f"{sshots_dir}/Sc_1LLTest_B_097.png",
    f"{sshots_dir}/Sc_1LLTest_B_098.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_1LLTest_B_099.png",
    f"{sshots_dir}/Sc_1LLTest_B_100.png",
    f"{sshots_dir}/Sc_1LLTest_B_101.png",
    f"{sshots_dir}/Sc_1LLTest_B_102.png",
    f"{sshots_dir}/Sc_1LLTest_B_103.png",
    f"{sshots_dir}/Sc_1LLTest_B_104.png",
    f"{sshots_dir}/Sc_1LLTest_B_105.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_1LLTest_B_106.png",
    f"{sshots_dir}/Sc_1LLTest_B_107.png",
    f"{sshots_dir}/Sc_1LLTest_B_108.png",
    f"{sshots_dir}/Sc_1LLTest_B_109.png",
    f"{sshots_dir}/Sc_1LLTest_B_110.png",
    f"{sshots_dir}/Sc_1LLTest_B_111.png",
    f"{sshots_dir}/Sc_1LLTest_B_112.png",
    f"{sshots_dir}/Blank.png",
]

In [80]:
len(imlist)

128

In [81]:
save_dir = sshots_dir

In [82]:
for i in range(len(imlist)):
    im = Image.open(imlist[i])
    if 'Blank' in imlist[i]:
        if "_A_" in imlist[0]:
            fn = f"{save_dir}/screen_A_{i+1:03d}.png"
        elif "_B_" in imlist[0]:
            fn = f"{save_dir}/screen_B_{i+1:03d}.png"
        im.save(fn,'PNG')
    else:
        im_mat = np.array(im)
        im_mat[im_mat[:,:,0] != 128] = 0
        im_mat[im_mat[:,:,1] != 128] = 0
        im_mat[im_mat[:,:,2] != 128] = 0
        #plt.imshow(im_mat)
        im_mat[im_mat[:,:,0] == 128] = -1
        im_mat[im_mat[:,:,1] == 128] = -1
        im_mat[im_mat[:,:,2] == 128] = -1
        #plt.imshow(im_mat)
        img2ar = Image.fromarray(im_mat.astype('uint8'), 'RGB')
        if "_A_" in imlist[i]:
            fn = f"{save_dir}/screen_A_{i+1:03d}.png"
        elif "_B_" in imlist[i]:
            fn = f"{save_dir}/screen_B_{i+1:03d}.png"
        img2ar.save(fn,'PNG')


#This will make the images but you should go over them individually to make sure that there are no stray pixels..else it will mess you pRF models
# The manual editing is required because of the complex visual stimuli.

In [51]:
# #edited_sshots_dir = f"{out_dir}/edited-screenshots"
# for i, s in enumerate(sorted(glob.glob(f"{save_dir}/screen*.png"))):
#     print(i, s)
#     im = Image.open(s)
#     im.save(f"{sshots_dir}/screen{i+65:03d}.png")

In [83]:
concat_sshot_list = []
for part in ["A", "B"]:
    img_list = sorted(glob.glob(f"{save_dir}/screen_{part}_*.png"))
    blanks_to_pad = 138-len(img_list)
    img_list.extend([img_list[-1]] * blanks_to_pad)
    concat_sshot_list.extend(img_list)
    print(len(imlist), len(concat_sshot_list))


128 138
128 276


In [84]:
biglist = [np.array([]) for _ in range(len(concat_sshot_list))]

#This flips the images around to make it like how popeye uses it
    
for i in range(0,len(concat_sshot_list)):
    im_read = cv.imread(concat_sshot_list[i])
    im_rgb = cv.cvtColor(im_read, cv.COLOR_BGR2RGB)
    im_gr = cv.cvtColor(im_rgb, cv.COLOR_BGR2GRAY)
    # im_grd = np.divide(im_gr,255)
    print(concat_sshot_list[i])
    ret,thresh = cv.threshold(im_gr,0,1,cv.THRESH_BINARY_INV)
    biglist[i] = thresh

bigstack = np.dstack((biglist))
pickle.dump(bigstack, open(f"{out_dir}/bigstack-{sub}-AB.pkl", 'wb'))


/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/Screenshots/screen_A_001.png
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/Screenshots/screen_A_002.png
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/Screenshots/screen_A_003.png
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/Screenshots/screen_A_004.png
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/Screenshots/screen_A_005.png
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/Screenshots/screen_A_006.png
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/Screenshots/screen_A_007.png
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/Screenshots/screen_A_008.png
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/Screenshots/screen_A_009.png
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/Screenshots/screen_A_010.png
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/Screen

/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/Screenshots/screen_A_109.png
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/Screenshots/screen_A_110.png
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/Screenshots/screen_A_111.png
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/Screenshots/screen_A_112.png
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/Screenshots/screen_A_113.png
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/Screenshots/screen_A_114.png
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/Screenshots/screen_A_115.png
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/Screenshots/screen_A_116.png
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/Screenshots/screen_A_117.png
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/Screenshots/screen_A_118.png
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/Screen

/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/Screenshots/screen_B_070.png
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/Screenshots/screen_B_071.png
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/Screenshots/screen_B_072.png
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/Screenshots/screen_B_073.png
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/Screenshots/screen_B_074.png
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/Screenshots/screen_B_075.png
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/Screenshots/screen_B_076.png
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/Screenshots/screen_B_077.png
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/Screenshots/screen_B_078.png
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/Screenshots/screen_B_079.png
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/Screen

In [85]:
glob.glob(f"{out_dir}/bigstack*.pkl")

['/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/bigstack-LL-B.pkl',
 '/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/bigstack-LL.pkl',
 '/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/bigstack-LL-AB.pkl']

## Mask manipulation

In [8]:
print(prf_mask_file)
!ls {prf_mask_file}

/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/sub-LL_ses-20210810_mask.nii.gz
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/sub-LL_ses-20210810_mask.nii.gz


In [12]:
#m = nib.load(f"{out_dir}/cortex_mask_funcspace.nii.gz")
m = nib.load(prf_mask_file)
m_chop = m.get_fdata().copy()
m_chop[:, 70:, :] = 0
chopped_mask_img = nib.Nifti1Image(m_chop, m.affine, m.header)
#chopped_mask_path = f"{out_dir}/sub-{sub}_ses-{ses}_desc-cortex-mask-back40_roi.nii.gz"
chopped_mask_path = f"{out_dir}/sub-{sub}_ses-{ses}_desc-wholebrainmask-back_roi.nii.gz"
nib.save(chopped_mask_img, chopped_mask_path)

In [10]:
print(f"fsleyes {anat_file} {prf_ref_vol_path} {prf_mask_file} {chopped_mask_path}")


fsleyes /Users/smerdis/data/LGN/BIDS/LL_combined/sub-LL/ses-20210810/anat/sub-LL_ses-20210810_T1w.nii.gz /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/sub-LL_ses-20210810_refvol.nii.gz /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/sub-LL_ses-20210810_mask.nii.gz /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/sub-LL_ses-20210810_desc-wholebrainmask-back40_roi.nii.gz


## Look at prf results (prfs run on cluster)

In [8]:
!ls {out_dir}

20210831-111808
20210907-113646
20210908-081328
Screenshots
bigstack-LL-AB.pkl
bigstack-LL-B.pkl
bigstack-LL.pkl
ses-20210810
sub-LL_ses-20210810_desc-wholebrainmask-back40_roi.nii.gz
sub-LL_ses-20210810_desc-wholebrainmask-back_roi.nii.gz
sub-LL_ses-20210810_mask.nii.gz
sub-LL_ses-20210810_mcflirt_mean_trim.nii
sub-LL_ses-20210810_mcflirt_mean_trim_evenrunsonly.nii
sub-LL_ses-20210810_mcflirt_mean_trim_oddevenconcat.nii
sub-LL_ses-20210810_mcflirt_mean_trim_oddrunsonly.nii
sub-LL_ses-20210810_refvol.json
sub-LL_ses-20210810_refvol.nii.gz


In [10]:
#main_analysis_dir = "/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/cluster_bin"
#prf_dir = f"{main_analysis_dir}/20210831-100746"
#prf_dir = f"{out_dir}/20210831-111808" # B runs (evens) only with bigstack B screenshots
#prf_dir = f"{out_dir}/20210907-113646" # AB concat (mean of odds, mean of evens)
prf_dir = f"{out_dir}/20210908-081328" # AB concat, 50k voxels (posterior 2/3rds of brain)

In [11]:
!ls {prf_dir}

bar.pkl
info.txt
sub-LL_ses-20210810_desc-prfmask_space-func_map.nii.gz
sub-LL_ses-20210810_desc-prfrho_space-func_map.nii.gz
sub-LL_ses-20210810_desc-prfrsq_space-func_map.nii.gz
sub-LL_ses-20210810_desc-prfsig_space-func_map.nii.gz
sub-LL_ses-20210810_desc-prftheta_space-func_map.nii.gz
sub-LL_ses-20210810_desc-prfxx_space-func_map.nii.gz
sub-LL_ses-20210810_desc-prfyy_space-func_map.nii.gz


In [12]:
prf_outputs = glob.glob(f"{prf_dir}/sub-{sub}*desc-prf[!mask]*map*")
print("\n".join(prf_outputs))

/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/20210908-081328/sub-LL_ses-20210810_desc-prfyy_space-func_map.nii.gz
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/20210908-081328/sub-LL_ses-20210810_desc-prftheta_space-func_map.nii.gz
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/20210908-081328/sub-LL_ses-20210810_desc-prfxx_space-func_map.nii.gz
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/20210908-081328/sub-LL_ses-20210810_desc-prfrho_space-func_map.nii.gz
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/20210908-081328/sub-LL_ses-20210810_desc-prfrsq_space-func_map.nii.gz


In [13]:
files_volspace = [x for x in prf_outputs if ('space-anat' not in x) \
        and (('rsq' in x) or ('rho' in x) or ('theta' in x)) \
        and 'thresh' not in x
]
print('\n'.join(files_volspace))
print(f"fsleyes {anat_file} {prf_ref_vol_path} {' '.join(files_volspace)}")

/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/20210908-081328/sub-LL_ses-20210810_desc-prftheta_space-func_map.nii.gz
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/20210908-081328/sub-LL_ses-20210810_desc-prfrho_space-func_map.nii.gz
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/20210908-081328/sub-LL_ses-20210810_desc-prfrsq_space-func_map.nii.gz
fsleyes /Users/smerdis/data/LGN/BIDS/LL_combined/sub-LL/ses-20210810/anat/sub-LL_ses-20210810_T1w.nii.gz /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/sub-LL_ses-20210810_refvol.nii.gz /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/20210908-081328/sub-LL_ses-20210810_desc-prftheta_space-func_map.nii.gz /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/20210908-081328/sub-LL_ses-20210810_desc-prfrho_space-func_map.nii.gz /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/20210908-081328/sub-LL_ses-20210810_desc-prfrsq_space-func_

#### Threshold by rsq

In [90]:
rsq_file = [f for f in prf_outputs if 'desc-prfrsq' in f and 'thresh' not in f][0]

for output in glob.glob(f"{prf_dir}/sub-{sub}*desc-prf[!mask]*map*"):
    if 'thresh-' not in output and 'space-func' in output:
        print(output)
        _ = utils.threshold(output, rsq_file, .05, prf_dir)

/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/20210907-113646/sub-LL_ses-20210810_desc-prfyy_space-func_map.nii.gz
Thresholding /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/20210907-113646/sub-LL_ses-20210810_desc-prfyy_space-func_map.nii.gz by /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/20210907-113646/sub-LL_ses-20210810_desc-prfrsq_space-func_map.nii.gz at thresh 0.05...
 (128, 128, 19) (128, 128, 19) 310272 1024
prfrsq
['sub-LL', 'ses-20210810', 'desc-prfyy', 'space-func', 'thresh-prfrsq-0.05', 'map.nii.gz']
sub-LL_ses-20210810_desc-prfyy_space-func_thresh-prfrsq-0.05_map.nii.gz
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/20210907-113646/sub-LL_ses-20210810_desc-prftheta_space-func_map.nii.gz
Thresholding /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/20210907-113646/sub-LL_ses-20210810_desc-prftheta_space-func_map.nii.gz by /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/2

#### Threshold by ecc (to filter out huge eccentricities that are a result of improper fits)

In [14]:
ecc_file = [f for f in prf_outputs if 'desc-prfrho' in f and 'thresh' not in f][0]
max_ecc = 12
for output in glob.glob(f"{prf_dir}/sub-{sub}*desc-prf[!mask]*map*"):
    _ = utils.threshold(output, ecc_file, max_ecc, prf_dir, how='less')
#     if f"thresh-prfrsq-0.20" not in output and f"thresh-prfrho" not in output:
#         #_ = threshold(output, ecc_file, max_ecc, prf_dir, how='less')
#         #_ = threshold(output, ecc_file, max_ecc, prf_dir, how='less')
#         print(output)

Thresholding /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/20210908-081328/sub-LL_ses-20210810_desc-prfyy_space-func_map.nii.gz by /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/20210908-081328/sub-LL_ses-20210810_desc-prfrho_space-func_map.nii.gz at thresh 12.00...
 (128, 128, 19) (128, 128, 19) 28133 23064
prfrho
['sub-LL', 'ses-20210810', 'desc-prfyy', 'space-func', 'thresh-prfrho-12.00', 'map.nii.gz']
sub-LL_ses-20210810_desc-prfyy_space-func_thresh-prfrho-12.00_map.nii.gz
Thresholding /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/20210908-081328/sub-LL_ses-20210810_desc-prftheta_space-func_map.nii.gz by /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/20210908-081328/sub-LL_ses-20210810_desc-prfrho_space-func_map.nii.gz at thresh 12.00...
 (128, 128, 19) (128, 128, 19) 28133 23064
prfrho
['sub-LL', 'ses-20210810', 'desc-prftheta', 'space-func', 'thresh-prfrho-12.00', 'map.nii.gz']
sub-LL_ses-20210810_desc-prftheta

In [11]:
files = [x for x in glob.glob(f"{prf_dir}/sub-{sub}*desc-prf[!mask]*map*") if ('space-anat' not in x)
                      and (('desc-prfrsq' in x) or ('desc-prfrho' in x) or ('desc-prftheta' in x))
        ] #and 'thresh-prfrsq' in x]
print('\n'.join(files))
print(f"fsleyes {anat_file} {prf_ref_vol_path} {' '.join(files)}")

/Users/smerdis/data/LGN/BIDS/JB_combined/derivatives/prf/sub-JB/20210604-142202/sub-JB_ses-20210402_desc-prftheta_space-func_thresh-prfrsq-0.10_map.nii.gz
/Users/smerdis/data/LGN/BIDS/JB_combined/derivatives/prf/sub-JB/20210604-142202/sub-JB_ses-20210402_desc-prfrho_space-func_thresh-prfrsq-0.10_map.nii.gz
/Users/smerdis/data/LGN/BIDS/JB_combined/derivatives/prf/sub-JB/20210604-142202/sub-JB_ses-20210402_desc-prfrsq_space-func_thresh-prfrsq-0.10_map.nii.gz
/Users/smerdis/data/LGN/BIDS/JB_combined/derivatives/prf/sub-JB/20210604-142202/sub-JB_ses-20210402_desc-prfrsq_space-func_map.nii.gz
/Users/smerdis/data/LGN/BIDS/JB_combined/derivatives/prf/sub-JB/20210604-142202/sub-JB_ses-20210402_desc-prfrho_space-func_map.nii.gz
/Users/smerdis/data/LGN/BIDS/JB_combined/derivatives/prf/sub-JB/20210604-142202/sub-JB_ses-20210402_desc-prftheta_space-func_map.nii.gz
fsleyes /Users/smerdis/data/LGN/BIDS/JB_combined/sub-JB/ses-20210402/anat/sub-JB_ses-20210402_T1w.nii.gz /Users/smerdis/data/LGN/BIDS/J

In [ ]:
for output in files:
    print(output)
    #if 'thresh-prfrsq' in output:
    utils.prf_to_anat(sub, f"{out_dir}/brain.nii.gz", output, f"{out_dir}/func2brain.mat", prf_dir) # my data
    #utils.prf_to_anat(sub, f"{out_dir}/brain.nii.gz", output, f"{out_dir}/func2brain_dev.mat", prf_dir) # dev's data

In [ ]:
files

In [ ]:
utils.freeview_prfs(sub, 'lh', prf_dir)

In [ ]:
utils.freeview_prfs(sub, 'rh', prf_dir)

In [ ]:
utils.freeview_prfs(sub, 'lh', f"{out_dir}/20210428-114114")

## Threshold ROI by pRF outputs and put into sub-ROIs

In [91]:
final_big_rois = ['/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_ses-20210511_desc-LLGN_roi.nii.gz',
 '/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_ses-20210511_desc-RLGN_roi.nii.gz']

In [ ]:
for roi in final_big_rois:
    utils.roi_stats({roi:load_img(roi)}, lgn_ref_img)
    if 'desc-L' in roi:
        high_roi, low_roi, threshold = utils.assign_roi_percentile(roi,
                                                           beta_MP_img,
                                                           80,
                                                           lgn_ref_img,
                                                           'L')
    elif 'desc-R' in roi:
        high_roi, low_roi, threshold = utils.assign_roi_percentile(roi,
                                                           beta_MP_img,
                                                           80,
                                                           lgn_ref_img,
                                                           'R')
    else:
        print('Error - not sure which hemisphere roi is in')
    utils.roi_stats({'high':load_img(high_roi), 'low':load_img(low_roi)}, prf_ref_img)